**BSP 2025-2026/1**

**Total of 20 marks**

**Names/Neptun Codes:**

**Marks attained for this part of the homework will count towards your final grade.**

**To attain (full) marks for a question, any answer that includes images or a piece of knowledge, which should be referenced/cited, needs to highlight the source (book, article or web address). You are encouraged to help each other, but identical solutions will not be graded.**

**Write your codes to this document for the full marks. Rename the document to: BSP25_05_TEAM_MONOGRAM**

## Task 1
(7 point)
Present an open source and state-of-the-art Python toolbox for biomedical signal analysis. Introduce the toolbox’s main components, features, and if applicable, its validation process. Demonstrate its usage with examples based on data you have downloaded. Finally, discuss the main limitations of the selected toolbox and suggest possible improvements.

## Task 2
### a) Calculate z-transform (0.5 point)

An $L$th-order average filter is a linear shift-invariant system that, for an input $x(n)$, produces the output

$$
y(n) = \frac{1}{L+1} \sum_{k=0}^{L} x(n-k)
$$

Find the frequency response of this system.

---

### b) Python routine (0.5 point)

Write your own Python routine from basic commands (i.e., without help from any Python toolbox) to plot the magnitude and phase response of the filter obtained in 1.a). Use normalized frequency \( f \) for the x-axis (i.e., divide by the sampling frequency), and plot in the range $( 0 \leq f \leq 1 )$.

**Hint:** Note that $( \omega = 2\pi f )$. You will need to define \( f \) (with enough elements to get a smooth result), then substitute it into the equations to compute and plot the magnitude and phase response.

---

### c) Plot signal (0.5 point)

Download the `complex.mat` file. In the `names` fields, you can check the recorded parameters. Plot the signals on a common plot.

**Hint:** You can use your own function, but remember to correct for the base and gain. Make sure to label axes and use the correct units.

---

### d) Filter signal (0.5 point)

Use the filter designed in 1.b) on the central venous pressure (CVP) signal. The signal is provided in the `Complex_signal.mat`, `mgh119m.hea` and `mgh119m.info` files. Plot your result.

**Hint:** You used the `filter(b, a, signal)` function in the previous assignment.

---

### e) Filter in physical form (0.5 point)

Write down the underlying difference equation of the filter from Task 2 in the following form:

$$
y(n) + a_1 y(n-1) + \dots + a_k y(n-k) = b_0 x(m) + b_1 x(m-1) + \dots + b_l x(m-l)
$$

---

### f) Signal-flow diagram (0.5 point)

Draw the signal-flow diagram for the realization of the filter (using adders, multipliers, and delay elements).

---

### g) Pole-zero diagram (0.5 point)

Draw the pole-zero diagram for the filter. Is the system stable? Why or why not? You may use the function `pzmap`, but also show how the pole-zero map is related to the transfer function.

### h) Plot ECG Lead II in Time and Frequency Domain (0.5 point)

Take the data from the second ECG lead and plot it in both the time and frequency domains. Answer the following questions:
- What is the bpm (beats per minute)?
- What is the breathing rate?
- What is the frequency of the powerline noise?

---

### i) Filter Powerline Noise with a Butterworth Filter (1 point)

Design a Butterworth filter to remove the powerline noise. Check the quality of the filter using `freqz()`. Plot both the filtered signal and its frequency spectrum.

**Hint:** Use the `butter` function to obtain the filter parameters \( b \) and \( a \). If needed, refer to `doc butter` for more details.

### Task 3 (3 point)

Choose one of the two tasks uploaded to the site. One task involves implementing a Wiener filter, while the other focuses on synchronized averaging. Implement in Python the methodologies outlined in the provided `WienerFilter.pdf` and `SynchronizedAveraging.pdf`.

### Task 4 (5 point)

Using the pyPPG toolbox’s prefiltering as a reference, design an equivalent FIR filter for systolic peak detection.
1. Apply the designed FIR filter on the example signal provided by the toolbox.
2. Plot the peak detection results for the IIR filter (from the toolbox) and your FIR filter.
3. Generate and compare the Bode diagram and pole-zero (PZ) map for both the IIR and FIR filters.
4. Provide the transfer functions for both the IIR and FIR filters.
5. Based on the plots (Bode diagram, PZ-map, and peak detection results), explain the differences between the IIR and FIR filters in terms of their frequency response, stability, and peak detection accuracy.